In [25]:
library(Seurat)
library(openxlsx)
library(readxl)
library(Matrix)
library(readr)

In [3]:
#Need to find out if all duplicates are nonzeros
#Adams
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/Adams")
#Upload matrices
data_adams_matrix <- readMM("data_adams_matrix.mtx")
#Upload original gene list
data_adams_genes <- read.xlsx("data_adams_geneIDs_int_removed_manualedit.xlsx")
data_adams_cellIDs <- read.delim("data_adams_cellIDs.tsv")
rownames(data_adams_matrix) <- data_adams_genes$New
colnames(data_adams_matrix) <- data_adams_cellIDs$cellIDs

In [4]:
#Check and extract duplicates
#Find sums of duplicate genes (from manual edit)
duplicate_genes <- duplicated(rownames(data_adams_matrix))
cat("Duplicate gene IDs:\n")
print(rownames(data_adams_matrix)[duplicate_genes])
duplicated <- rownames(data_adams_matrix)[duplicate_genes]

Duplicate gene IDs:
 [1] "GABPB1"         "RRM2"           "TMC6"           "MT-TP"         
 [5] "LYPLAL1-DT"     "SIMALR"         "LINC01291"      "FAM27C"        
 [9] "UBE2R2-AS1"     "ROPN1L"         "TNPO1-DT"       "KHDRBS2"       
[13] "GCNT4"          "SIGLEC12"       "CFAP298-TCP10L" "SIGLEC5"       
[17] "GPR89B"         "TBC1D3B"        "PMS2P10"        "CCDC92B"       
[21] "IGSF9B"         "ZNF724"         "NBPF13P"        "BCRP3"         
[25] "TSTD3"          "LINC01819"      "LINC00499"      "MILIP"         
[29] "KRT6A"          "SEPTIN4"        "SPTBN1-AS1"     "LINC02814"     
[33] "LINC02149"      "LINC01115"      "PSG9"           "PKNOX2-DT"     


In [10]:
#Filter zeros
row_sums <- rowSums(data_adams_matrix)
names_with_zeros <- names(row_sums[row_sums == 0])
length(names_with_zeros)

[1] 20

In [11]:
names_with_zeros

[1] "ENSG00000225570" "ENSG00000197476" "IGHV3-62"        "ENSG00000257994"
 [5] "ENSG00000279657" "RPSAP62"         "IGHV1OR16-4"     "IGLJ2"          
 [9] "RNU6-1135P"      "ENSG00000279042" "OR4A21P"         "ENSG00000248137"
[13] "ENSG00000267400" "GAGE12H"         "ENSG00000266196" "RN7SL445P"      
[17] "ENSG00000275480" "ENSG00000207243" "ENSG00000250243" "OR13C7"

In [12]:
filtered_data_adams_matrix <- data_adams_matrix[row_sums != 0, ]

In [19]:
str(filtered_data_adams_matrix)
str(data_adams_matrix)

Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:548760947] 5 16 29 32 47 53 71 72 73 75 ...
  ..@ j       : int [1:548760947] 0 0 0 0 0 0 0 0 0 0 ...
  ..@ Dim     : int [1:2] 42608 239705
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:42608] "TSPAN6" "TNMD" "DPM1" "SCYL3" ...
  .. ..$ : chr [1:239705] "001C_AAACCTGCATCGGGTC" "001C_AAACCTGTCAACACCA" "001C_AAACCTGTCACAGTAC" "001C_AAACCTGTCTGTCTAT" ...
  ..@ x       : num [1:548760947] 2 2 1 1 1 1 1 1 3 1 ...
  ..@ factors : list()
Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:548760947] 5 16 29 32 47 53 71 72 73 75 ...
  ..@ j       : int [1:548760947] 0 0 0 0 0 0 0 0 0 0 ...
  ..@ Dim     : int [1:2] 42628 239705
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:42628] "TSPAN6" "TNMD" "DPM1" "SCYL3" ...
  .. ..$ : chr [1:239705] "001C_AAACCTGCATCGGGTC" "001C_AAACCTGTCAACACCA" "001C_AAACCTGTCACAGTAC" "001C_AAACCTGTCTGTCTAT" ...
  ..@ x       : num [1:548760947] 2 2 1 1 1 1 1 1 3 1 .

In [13]:
#Concatenating replicates
filtered_data_adams_matrix <- as(filtered_data_adams_matrix, "dgCMatrix")

'as(<dgTMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").



In [14]:
gene_names <- rownames(filtered_data_adams_matrix)
unique_gene_names <- unique(gene_names)
gene_factors <- factor(gene_names, levels = unique_gene_names)

In [15]:
aggregated_counts <- rowsum(as.matrix(filtered_data_adams_matrix), group = gene_factors)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 76.1 GiB”


In [16]:
str(aggregated_counts)

 num [1:42572, 1:239705] 0 0 0 0 0 2 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:42572] "TSPAN6" "TNMD" "DPM1" "SCYL3" ...
  ..$ : chr [1:239705] "001C_AAACCTGCATCGGGTC" "001C_AAACCTGTCAACACCA" "001C_AAACCTGTCACAGTAC" "001C_AAACCTGTCTGTCTAT" ...


In [19]:
#Load metadata and dim reductions
data_adams_metadata <- read_csv("data_adams_metadata_final.csv")

New names:
• `` -> `...1`
• `...1` -> `...2`
Rows: 239705 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (13): ...2, cells, lineage, aetiology, condition, subclass, study, tissu...
dbl  (5): ...1, nCount_RNA, nFeature_RNA, percent.mt, age

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [20]:
data_adams_metadata <- as.data.frame(data_adams_metadata)

In [21]:
rownames(data_adams_metadata) <- data_adams_metadata$`...2`

In [22]:
data_adams_metadata$`...2` <- NULL
data_adams_metadata$`...1` <- NULL

In [27]:
class(aggregated_counts)

[1] "matrix" "array"

In [28]:
aggregated_counts <- as(aggregated_counts, "dgCMatrix")

In [31]:
class(aggregated_counts)

[1] "dgCMatrix"
attr(,"package")
[1] "Matrix"

In [29]:
writeMM(aggregated_counts,file='data_adams_aggregated_counts.mtx')

NULL

In [32]:
#The process resulted in correct aggregation (refer to excel sheet)
#Create Seurat Object with the new gene list
data_adams_seurat <- CreateSeuratObject(counts = aggregated_counts, meta.data = data_adams_metadata)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [33]:
data_adams_seurat

An object of class Seurat 
42572 features across 239705 samples within 1 assay 
Active assay: RNA (42572 features, 0 variable features)
 1 layer present: counts

In [34]:
save(data_adams_seurat, file = "data_adams_seurat_final.RData")